In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from sklearn import metrics
import tabulate
import itertools
import umap
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import goatools
from goatools.anno.gaf_reader import GafReader
import Bio.UniProt.GOA as GOA
import gget

# locals
import utils as ut
reload(ut)

2022-12-21 10:08:50.711431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 10:08:50.825848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2022-12-21 10:08:50.825869: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-21 10:08:50.851703: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-21 10:08:52.1955

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [4]:
query = ['H2-EB1', 'SELENOP', 'C1QB', 'C1QA', 'CSF1R', 'CCDC152', 'RETNLA', 'CD209F', 'WFDC17', 'C1QC', 'CTSB', 'H2-AB1', 'CD74', 'CTSS', 'CCL6', 'RAB7B', 'PF4', 'LY86', 'SAMHD1', 'APOBR']
# db = 'WikiPathways_2019_Mouse'
# db = 'KEGG_2019_Mouse'
# db = 'ontology'
db = 'celltypes'
ef = gget.enrichr(query, database=db)

pd.set_option('display.max_colwidth', 200)


ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

Wed Dec 21 10:09:36 2022 INFO Performing Enichr analysis using database PanglaoDB_Augmented_2021. 
    Please note that there might a more appropriate database for your application. 
    Go to https://maayanlab.cloud/Enrichr/#libraries for a full list of supported databases.
    


,path_name,p_val,overlapping_genes
0,Kupffer Cells,4.362463e-11,"[C1QB, CSF1R, C1QA, LY86, APOBR, CTSS, C1QC]"
1,Microglia,1.194536e-10,"[C1QB, CSF1R, C1QA, LY86, APOBR, CTSS, PF4]"
2,Macrophages,7.176768e-10,"[CSF1R, CD74, SELENOP, LY86, APOBR, RAB7B, CTSS]"
3,Spermatozoa,7.851895e-10,"[C1QB, CSF1R, C1QA, CD74, CTSS, C1QC]"
4,Langerhans Cells,1.728002e-09,"[CSF1R, C1QA, CD74, LY86, APOBR, CTSS]"
5,Osteoclasts,2.564373e-07,"[C1QB, CSF1R, C1QA, C1QC, CTSB]"
6,Dendritic Cells,1.273307e-06,"[CD74, LY86, APOBR, RAB7B, CTSS]"
7,Red Pulp Macrophages,3.921006e-06,"[CSF1R, C1QA, LY86, CTSS]"
8,Satellite Glial Cells,4.520450e-06,"[C1QB, CSF1R, C1QA, C1QC]"
9,Alveolar Macrophages,6.114816e-06,"[CSF1R, LY86, APOBR, CTSS]"


In [3]:
?gget.enrichr

Signature:
gget.enrichr(
    genes,
    database,
    ensembl=False,
    plot=False,
    figsize=(10, 10),
    ax=None,
    json=False,
    save=False,
)
Docstring:
Perform an enrichment analysis on a list of genes using Enrichr (https://maayanlab.cloud/Enrichr/).

Args:
- genes       List of Entrez gene symbols to perform enrichment analysis on, passed as a list of strings, e.g. ['PHF14', 'RBM3', 'MSL1', 'PHF21A'].
              Set 'ensembl = True' to input a list of Ensembl gene IDs, e.g. ['ENSG00000106443', 'ENSG00000102317', 'ENSG00000188895'].
- database    Database to use as reference for the enrichment analysis.
              Supported shortcuts (and their default database):
              'pathway' (KEGG_2021_Human)
              'transcription' (ChEA_2016)
              'ontology' (GO_Biological_Process_2021)
              'diseases_drugs' (GWAS_Catalog_2019)
              'celltypes' (PanglaoDB_Augmented_2021)
              'kinase_interactions' (KEA_2015)
              or an